In [ ]:

Full_load_path = "Files/Bronze/salesdata_raw"
Incremental_Load = "Files/Bronze/sales_incremental_data"
silver_layer_path = "Files/Silver/Sales_data"


In [ ]:

# Read Full Load and Incremental Load Data
full_load_df = spark.read.format("csv").option("header", True).load(Full_load_path)
incremental_load_df = spark.read.format("csv").option("header", True).load(Incremental_Load)


In [ ]:

from pyspark.sql import types as T

def clean_incremental_data(incremental_load_df):
    incremental_load_df = incremental_load_df.withColumn('SALES_ID', incremental_load_df['SALES_ID'].cast(T.LongType()))
    incremental_load_df = incremental_load_df.withColumn('SALES_DATE', incremental_load_df['SALES_DATE'].cast(T.TimestampType()))
    incremental_load_df = incremental_load_df.withColumn('UNIT_PRICE', incremental_load_df['UNIT_PRICE'].cast(T.LongType()))
    incremental_load_df = incremental_load_df.withColumn('QUANTITY_SOLD', incremental_load_df['QUANTITY_SOLD'].cast(T.LongType()))
    incremental_load_df = incremental_load_df.withColumn('TOTAL_AMOUNT', incremental_load_df['TOTAL_AMOUNT'].cast(T.DoubleType()))
    incremental_load_df = incremental_load_df.withColumn('DISCOUNT', incremental_load_df['DISCOUNT'].cast(T.DoubleType()))
    incremental_load_df = incremental_load_df.withColumn('LAST_UPDATED_DATE', incremental_load_df['LAST_UPDATED_DATE'].cast(T.TimestampType()))
    return incremental_load_df

incremental_load_df_clean = clean_incremental_data(incremental_load_df)
display(incremental_load_df_clean)


In [ ]:

def clean_full_data(full_load_df):
    full_load_df = full_load_df.withColumn('SALES_ID', full_load_df['SALES_ID'].cast(T.LongType()))
    full_load_df = full_load_df.withColumn('SALES_DATE', full_load_df['SALES_DATE'].cast(T.TimestampType()))
    full_load_df = full_load_df.withColumn('UNIT_PRICE', full_load_df['UNIT_PRICE'].cast(T.LongType()))
    full_load_df = full_load_df.withColumn('LAST_UPDATED_DATE', full_load_df['LAST_UPDATED_DATE'].cast(T.TimestampType()))
    full_load_df = full_load_df.withColumn('QUANTITY_SOLD', full_load_df['QUANTITY_SOLD'].cast(T.LongType()))
    full_load_df = full_load_df.withColumn('DISCOUNT', full_load_df['DISCOUNT'].cast(T.DoubleType()))
    full_load_df = full_load_df.withColumn('TOTAL_AMOUNT', full_load_df['TOTAL_AMOUNT'].cast(T.DoubleType()))
    return full_load_df

full_load_df_clean = clean_full_data(full_load_df)
display(full_load_df_clean)


In [ ]:

# Combine both raw and incremental files
Combined_df = full_load_df_clean.unionByName(incremental_load_df_clean)
display(Combined_df)


In [ ]:

# Deduplicate the combined data based on SALES_ID
Combined_df_deduplicated = Combined_df.dropDuplicates(["SALES_ID"])
display(Combined_df_deduplicated)
